Can follow some code in timm's dem merging function.
https://github.com/accs-uaa/beringian-vegetation/blob/master/package_GeospatialProcessing/mergeElevationTiles.py

Recommended workflow for geoprocessing:
- import arcpy
- set environments
- create lists of gis objects
- iterate through list elements
- execute geoprocessing tools

Geoprocessing steps in this script
1. create a set of folders and gdb for each region under a master AKSSF folder
2. loop through regions and grids or feature classes (e.g. catchments or flowlines)
3. merge grids or feature classes
4. save in folder/gdb with consistent name (e.g. fac_merge)

These seamed grids will enable us to use zonal statistics with the selected catchments
and watersheds to get our environmental variables for modeling.

# step 1
# Create folders for storing outputs

#import modules
import os
import arcpy

#user inputs
baseFolder = "W:\\GIS\\"
projectName = "AKSSF"

#define folder paths
ciFolder = baseFolder + "\\" + projectName + "\\"+ "Cook_Inlet"
pwsFolder = baseFolder + "\\" + projectName + "\\"+ "Prince_William_Sound"
bbFolder = baseFolder + "\\" + projectName + "\\"+ "Bristol_Bay"
crFolder = baseFolder + "\\" + projectName + "\\"+ "Copper_River"
kodFolder = baseFolder + "\\" + projectName + "\\"+ "Kodiak"

#create list of folder paths
folders = [ciFolder, pwsFolder, bbFolder, crFolder, kodFolder]

#check if project folder exists
if not os.path.exists(baseFolder + projectName):
    #Create folder if it does not exist
    for folder in folders:
        os.makedirs(folder)
        print("Created: " + folder)
else:
    #if project folder exists, print error message
    print (projectName + " exists. Please provide a new project name.")

In [ ]:
# step 2
# merge catchments and vaa tables
#This code chunk is for NHDPlus regions -- Cook Inlet and Copper River

import arcpy
import os

baseFolder = "W:\\GIS\\"
region = "Copper_River"

#NHD folders
ciNHD = baseFolder + "NHDPlus\\CookInlet_20201216"
crNHD = baseFolder + "NHDPlus\\CopperRiver_20210408"

arcpy.env.workspace = crNHD
gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
print(gdbs)

Cats = []
VAA = []
for gdb in gdbs:
    arcpy.env.workspace = gdb
    catpath = os.path.join(arcpy.env.workspace, "NHDPlus\\NHDPlusCatchment")
    Cats.append(catpath)
    vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
    VAA.append(vaapath)

arcpy.CreateFileGDB_management(r"W:/GIS/AKSSF/" + region, region + ".gdb")
arcpy.env.workspace =  r"W:/GIS/AKSSF/" + region + "//" + region + ".gdb"
arcpy.env.outputCoordinateSystem = arcpy.Describe(Cats[1]).spatialReference
arcpy.env.overwriteOutput = True

arcpy.management.Merge(Cats, r"W:/GIS/AKSSF/" + region + "//" + region + ".gdb/cats_merge")
arcpy.management.Merge(VAA, r"W:/GIS/AKSSF/" + region + "//" + region + ".gdb/vaa_merge")


# Merge catchments and streams for Bristol Bay

This code chunk is for Bristol Bay, which had several processing units created
with synthetic networks from TauDEM. The gridcodes all start from 1 so are not unique
when combining the different VPUS. To create unique IDs across the VPUs,
need to make a new catID with a prefix for each VPU: 10, 20, 30, 40, 50.
Note that gridcode = LINKNO in the stream reach feature class.
Also for the streams fc that we are using to create watersheds, the upstream
links must be fixed so they are unique as well.

For Kodiak and Prince William Sound, can just copy over catchments and streams since
only one feature class for each.

In [1]:
import arcpy
import os

baseFolder = "W:\\GIS\\"
ds = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs"
region = "Bristol_Bay"

arcpy.env.workspace = ds
arcpy.env.overwriteOutput = True
cats = arcpy.ListFeatureClasses('Catchments*')
streams = arcpy.ListFeatureClasses('Stream*')

#For Bristol Bay - remove feature classes with Pws
cats = [x for x in cats if 'Pws' not in x]
streams = [x for x in streams if 'Pws' not in x]

#get full path names onto lists
cats = [ds + "\\" + x for x in cats]
streams = [ds + "\\" + x for x in streams]

#add a new field with the VPU prefix to the gridcode.
for cat in cats:
    if len(arcpy.ListFields(cat, "catID")) > 0:
        arcpy.DeleteField_management(cat, "catID")
        arcpy.AddField_management(cat, "catID", "LONG")
    vpu = cat[61:63]
    print(vpu)
    options = {'Eg': 1000000, 'La': 2000000, 'Na': 3000000, 'Nu': 4000000, 'To': 5000000}
    if vpu in options:
        prefix = options[vpu]
    else:
        print("vpu not found")
    arcpy.management.CalculateField(cat, "catID", 'prefix + !gridcode!')

for stream in streams:
    print(stream)
    if len(arcpy.ListFields(stream, "catID")) > 0:
        arcpy.DeleteField_management(stream, ["catID", "upCatID1", "upCatID2"])
        arcpy.management.AddFields(stream,[["catID", "LONG"], ["upCatID1", "LONG"], ["upCatID2", "LONG"]])
    vpu = stream[65:67]
    print(vpu)
    # options = {'Eg': 1000000, 'La': 2000000, 'Na': 3000000, 'Nu': 4000000, 'To': 5000000}
    if vpu in options:
        prefix = options[vpu]
    else:
        print("vpu not found")
    arcpy.management.CalculateField(stream, "catID", 'prefix + !LINKNO!')
    arcpy.management.CalculateField(stream, "upCatID1", 'prefix + !USLINKNO1!')
    arcpy.management.CalculateField(stream, "upCatID2", 'prefix + !USLINKNO2!')


print("Catchment feature classes to be merged: ")
print(cats)
print("Stream feature classes to be merged: ")
print(streams)

arcpy.env.outputCoordinateSystem = arcpy.Describe(cats[1]).spatialReference
arcpy.env.overwriteOutput = True
#arcpy.CreateFileGDB_management(baseFolder + "AKSSF\\" + region, region + ".gdb")
arcpy.env.workspace =  r"W:/GIS/AKSSF/" + region + ".gdb"

arcpy.management.Merge(cats, r"W:/GIS/AKSSF/" + region + "\\" + region + ".gdb/cats_merge")
arcpy.management.Merge(streams, r"W:/GIS/AKSSF/" + region + "\\" + region + ".gdb/streams_merge")

print("Merged feature classes saved to: " + arcpy.env.workspace)


Eg
La
Na
Nu
To
W:\GIS\AKSSF\AKSSF_Hydrography.gdb\TauDEM_Outputs\Stream_Network_Egegik_stream_reach
Eg
W:\GIS\AKSSF\AKSSF_Hydrography.gdb\TauDEM_Outputs\Stream_Network_Naknek_stream_reach
Na
W:\GIS\AKSSF\AKSSF_Hydrography.gdb\TauDEM_Outputs\Stream_Network_Nushagak_stream_reach
Nu
W:\GIS\AKSSF\AKSSF_Hydrography.gdb\TauDEM_Outputs\Stream_Network_Togiak_stream_reach
To
W:\GIS\AKSSF\AKSSF_Hydrography.gdb\TauDEM_Outputs\Stream_Network_Lakes_stream_reach
La
Catchment feature classes to be merged: 
['W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\Catchments_Egegik_diss', 'W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\Catchments_Lakes_diss', 'W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\Catchments_Naknek_diss', 'W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\Catchments_Nushagak_diss', 'W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\Catchments_Togiak_diss']
Stream feature classes to be merged: 
['W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs\\S

In [ ]:
# Copy feature classes from AKSSF Hydrography geodb to new region specific geodb
# for Kodiak and PWS. These will be used to create watersheds and run zonal
# stats for catchments and watersheds.

import arcpy
import os

# Create new geodb in each regional folder
baseFolder = "W:\\GIS\\"
region = "Prince_William_Sound"
# region = "Kodiak"

# TauDEM outputs
pwsNet = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs"
kodNet = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_StreamBurn"

arcpy.env.workspace = pwsNet
fcs = arcpy.ListFeatureClasses()
fcs = [os.path.join(arcpy.env.workspace, x) for x in fcs]
fcs = [x for x in fcs if "Pws" in x]
print(fcs)

# arcpy.CreateFileGDB_management(r"W:/GIS/AKSSF/" + region, region + ".gdb")
arcpy.env.workspace =  r"W:/GIS/AKSSF/" + region + "//" + region + ".gdb"
arcpy.env.outputCoordinateSystem = arcpy.Describe(fcs[0]).spatialReference
arcpy.env.overwriteOutput = True

# Copy feature classes over from AKSSF_Hydrography geodb
for fc in fcs:
    if "Cat" in fc:
        out_fc = "cats_merge"
    else:
        out_fc = "streams_merge"
    arcpy.CopyFeatures_management(fc, out_fc)

# Merge raster datasets by region
1. Create a regional boundary (maybe buffer by 10 km) for clipping statewide datasets and putting all grids into regional geodb
2. Elevation grids: clip 10m DEM for synthetic flow networks (ask Dustin for this) and merge elevation rasters for each NHDPlus VPU
3. Flow accumulation grids: copy and/or merge as needed by region
4. Use elevation grids to create slope grids for each region
5. Use elevation grids to create the aspect grid
6. Use elevation grids to create a grid of north facing cells (Dustin has this code ready)
7. Lake feature class: merge from NHPPlus VPUs or clip from regular NHD to region geodb
8. Copy glims glacier inventory to local AKSSF folder and clip and create a 1/0 grid of glacier cover
9. Copy NLCD to local AKSSF folder and clip create a 1/0 grid of wetland cover (Dustin should have this code in deshka repo)
10. Create raster of MODIS LCLD by year, copy, and clip to each regional folder

In [ ]:
# step 1, create regional boundaries with buffer for clipping grids
# dissolve on cats_merge and buffer by 10 km

import arcpy
import os

regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]
arcpy.env.overwriteOutput = True

for region in regions:
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb"
    cats = os.path.join(arcpy.env.workspace, "cats_merge")
    arcpy.Buffer_analysis(cats, "region_buf", "10 Kilometers", "FULL", "ROUND", "ALL")

In [4]:
# steps 2 and 3: merge elevation and flow accumulation rasters for regions with NHDPlus
#CURRENTLY NOT WORKING, MOSAIC WAS EMPTY FOR FAC AND NEVER RAN FOR ELEV.

import arcpy
import os

baseFolder = "W:\\GIS\\"
regions = ["Cook_Inlet", "Copper_River"]

#NHD folders
ciNHD = baseFolder + "NHDPlus\\CookInlet_20201216"
crNHD = baseFolder + "NHDPlus\\CopperRiver_20210408"

for region in regions:
    if region == "Cook_Inlet":
        arcpy.env.workspace = ciNHD
    else:
        arcpy.env.workspace = crNHD
    print("Processing " + region)
    flowacc_rasters = []
    elev_rasters = []
    raster_folders = arcpy.ListWorkspaces("*Rasters*")
    for folder in raster_folders:
        arcpy.env.workspace = folder
        rasters = arcpy.ListRasters()
        for raster in rasters:
            if raster == 'fac.tif':
                flowacc_raspath = os.path.normpath(os.path.join(folder, raster))
                flowacc_rasters.append(flowacc_raspath)
            if raster == 'elev_cm.tif':
                elev_raspath = os.path.normpath(os.path.join(folder, raster))
                elev_rasters.append(elev_raspath)
    dataset = flowacc_rasters[1]
    spatial_ref = arcpy.Describe(dataset).spatialReference
    outFolder = baseFolder + "AKSSF\\" + region
    print(outFolder)
    print(flowacc_rasters)
    print(elev_rasters)
    arcpy.MosaicToNewRaster_management(flowacc_rasters, outFolder, "fac_merge.tif", spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
    arcpy.MosaicToNewRaster_management(elev_rasters, outFolder, "elev_merge.tif", spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")

Processing Cook_Inlet
W:\GIS\AKSSF\Cook_Inlet
['W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020202\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020301\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020302\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020401\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020402\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020501\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020502\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020503\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020504\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020505\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020601\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020602\\fac.tif', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRaste

In [ ]:
# steps 2 and 3: clip elevation grid and copy flow accumulation raster for regions with synthetic networks
# (ask Dustin to do this or supply these layers)


In [ ]:
# steps 4-6, use elevation grids for each region to create slope, aspect, and north-facing grids
import arcpy
from arcpy import env
from arcpy.sa import *

baseFolder = "W:\\GIS\\AKSSF"
arcpy.env.workspace = baseFolder
region_folders = arcpy.ListWorkspaces()

for folder in region_folders:
    arcpy.env.workspace = folder
    elev_raster = folder + "elev_merge.tif"
    slope_raster = arcpy.sa.Slope(elev_raster, "DEGREE")
    slope_raster.save("slope")
    aspect_raster = arcpy.sa.Aspect(elev_raster)
    aspect_raster.save("aspect")
    north_raster = Con((aspect_raster >= 0) & (aspect_raster <= 45), 1, Con((aspect_raster <= 360) & (aspect_raster >= 315), 1, 0))
    north_raster.save("north")